In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\Ben\Desktop\Career Foundry Tasks\Instacart Basket Analysis 02-2023'

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_new_variables.pkl'))

In [4]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Because this is the full dataset and not a subset, all department ids are listed. The mean of the same department ids is different here than the subset because it is including all data points. The subset mean compared to the full dataset mean is quite different. More of a difference than I would have expected, for example department 14 mean is 19.2 in the subset but only 16.7 with the full data.

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

In [6]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours_of_the_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,195,...,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer


In [12]:
#Spending habits of the three types of customers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [14]:
#Average price spending flag
ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [16]:
ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spender_flag'] = 'Low spender'

In [17]:
ords_prods_merge.loc[ords_prods_merge['average_price'] >= 10, 'spender_flag'] = 'High spender'

In [18]:
ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_flag, dtype: int64

In [19]:
#Frequency flag
ords_prods_merge['median_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
ords_prods_merge.loc[ords_prods_merge['median_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [22]:
ords_prods_merge.loc[(ords_prods_merge['median_prior_order'] <= 20) & (ords_prods_merge['median_prior_order'] < 10), 'frequency_flag'] = 'Regular customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['median_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [24]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
NaN                       7208569
Non-frequent customer     3636437
Name: frequency_flag, dtype: int64

In [25]:
#Export dataframe
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_agg.pkl'))